In [48]:
from azureml.core import Workspace, Dataset
import pandas as pd
from io import StringIO

workspace = Workspace.from_config()
datastore = workspace.get_default_datastore()

# (Optional) Only if you haven't uploaded already!
data = """age,income,target
25,50000,0
30,60000,1
35,55000,0
40,65000,1"""
dataset_file = StringIO(data)
df = pd.read_csv(dataset_file)
df.to_csv("training_data.csv", index=False)
datastore.upload_files(["training_data.csv"], target_path="data/", overwrite=True)

# Create and register dataset
dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'data/training_data.csv'))
dataset = dataset.register(workspace=workspace, name='training_dataset', create_new_version=True)
print("Dataset registered successfully.")

Uploading an estimated of 1 files
Uploading training_data.csv
Uploaded training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Dataset registered successfully.


In [49]:
from azureml.core.compute import ComputeTarget

for name, ct in workspace.compute_targets.items():
    print(name, " -> ", ct.type)


Nobo  ->  ComputeInstance


In [50]:
from azureml.core import Environment, ScriptRunConfig, Experiment, ComputeTarget, Dataset
from azureml.core.conda_dependencies import CondaDependencies

workspace = Workspace.from_config()

# Set up environment (do this ONCE; reuse the same environment name after)
env = Environment(name="custom_environment")
dependencies = CondaDependencies.create(
    conda_packages=['scikit-learn', 'pandas', 'numpy', 'joblib'],
    pip_packages=['azureml-sdk']
)
env.python.conda_dependencies = dependencies
env.register(workspace=workspace)

# Make sure your compute target exists and is running
compute_target = ComputeTarget(workspace=workspace, name="Nobo")

In [52]:
from azureml.core import ScriptRunConfig

# Load dataset by its registered name
dataset = Dataset.get_by_name(workspace, name='training_dataset')

src = ScriptRunConfig(
    source_directory='.',   # Assumes train.py is in the same directory
    script='train.py',
    arguments=['--input_data', dataset.as_named_input('training_data')],
    compute_target=compute_target,
    environment=env
)

In [53]:
experiment = Experiment(workspace, 'my_experiment')
run = experiment.submit(src)
print("Run submitted!")

Run submitted!


In [55]:
run.wait_for_completion(show_output=True)

RunId: my_experiment_1763715496_c5be8e19
Web View: https://ml.azure.com/runs/my_experiment_1763715496_c5be8e19?wsid=/subscriptions/fd8d2a3b-f1ca-401a-aa76-86130cd5f908/resourcegroups/azureml1/workspaces/azuremlnew&tid=9fa6fe26-baee-451c-9182-76a5328ce10f

Execution Summary
RunId: my_experiment_1763715496_c5be8e19
Web View: https://ml.azure.com/runs/my_experiment_1763715496_c5be8e19?wsid=/subscriptions/fd8d2a3b-f1ca-401a-aa76-86130cd5f908/resourcegroups/azureml1/workspaces/azuremlnew&tid=9fa6fe26-baee-451c-9182-76a5328ce10f



{'runId': 'my_experiment_1763715496_c5be8e19',
 'target': 'Nobo',
 'status': 'Completed',
 'startTimeUtc': '2025-11-21T08:58:25.747548Z',
 'endTimeUtc': '2025-11-21T08:58:38.76579Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcdsi',
  '_azureml.ClusterName': 'Nobo',
  'ContentSnapshotId': '0b34a877-0740-4577-958c-ec5a1de9b322',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '69968155-b4df-49a1-948c-d0e26b7a438e'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'training_data', 'mechanism': 'Direct'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--input_data', 'DatasetConsumptionConfig:training_data'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'Nobo',
  'dataReferences': {},
  'data': {'training_data': {'dataLoca